In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
import numpy as np

# ========================================================================
# CONFIGURAZIONE STILE PROFESSIONALE - TEMA SCURO
# ========================================================================

plt.style.use('dark_background')

# Configurazione globale per aspetto professionale
plt.rcParams.update({
    # Figure
    'figure.facecolor': '#0a0a0a',
    'figure.edgecolor': '#0a0a0a',
    'figure.dpi': 150,
    
    # Assi
    'axes.facecolor': '#1a1a1a',
    'axes.edgecolor': '#00ff9d',
    'axes.linewidth': 1.5,
    'axes.grid': True,
    'axes.labelsize': 13,
    'axes.titlesize': 15,
    'axes.titleweight': 'bold',
    'axes.labelcolor': '#e0e0e0',
    
    # Griglia
    'grid.color': '#333333',
    'grid.linestyle': '--',
    'grid.linewidth': 0.5,
    'grid.alpha': 0.3,
    
    # Linee
    'lines.linewidth': 2.5,
    
    # Testo
    'text.color': '#e0e0e0',
    'font.size': 11,
    'font.family': 'sans-serif',
    'font.sans-serif': ['DejaVu Sans', 'Arial', 'Helvetica'],
    
    # Legenda
    'legend.facecolor': '#1a1a1a',
    'legend.edgecolor': '#00ff9d',
    'legend.framealpha': 0.9,
    'legend.fontsize': 10,
    
    # Ticks
    'xtick.color': '#b0b0b0',
    'ytick.color': '#b0b0b0',
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
})

# Palette colori professionale
COLORS = {
    'primary': '#00ff9d',      # Verde acqua brillante
    'secondary': '#00d4ff',    # Azzurro elettrico
    'accent': '#ff006e',       # Rosa acceso
    'warning': '#ffbe0b',      # Giallo oro
    'fill': '#00ff9d',         # Riempimento
    'curve': '#00d4ff',        # Curva principale
    'rect_edge': '#00ff9d',    # Bordo rettangoli
    'rect_fill': '#00ff9d',    # Riempimento rettangoli
}

print("✓ Configurazione stile professionale caricata")
print("✓ Tema: Dark Mode")
print("✓ Palette: Neon Tech")

✓ Configurazione stile professionale caricata
✓ Tema: Dark Mode
✓ Palette: Neon Tech


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter

# ========================================================================
# 1. CONFIGURAZIONE GENERALE E STILE
# ========================================================================

VIDEO_CONFIG = {
    'fps': 30,
    'dpi': 150,
    'duration_body': 12,    # Durata dell'animazione effettiva
    'duration_pause': 2,    # Pausa statica all'inizio (secondi)
    'res_width': 10,
    'res_height': 10
}

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
    "mathtext.fontset": "cm",
    "font.family": "serif",
    "figure.facecolor": "white",
    "axes.facecolor": "white",
    "axes.edgecolor": "black",
    "text.color": "black"
})

COLORS = {
    'func_line': '#003366',
    'rect_fill': '#A0C4FF',
    'rect_edge': '#003366',
    'true_area': '#F0F0F0',
    'error_line': '#B22222',
}

# ========================================================================
# 2. PARAMETRI MATEMATICI E SEQUENZA TEMPORALE
# ========================================================================

def f(x):
    return np.sin(x) + 0.5 * x

a, b = 0, 5
x_smooth = np.linspace(a, b, 1000)
y_smooth = f(x_smooth)
area_vera = np.trapezoid(y_smooth, x_smooth)

# Calcolo dei frame
fps = VIDEO_CONFIG['fps']
pause_frames = int(VIDEO_CONFIG['duration_pause'] * fps)
body_frames = int(VIDEO_CONFIG['duration_body'] * fps)
total_frames = pause_frames + body_frames

# Generazione n: costante all'inizio, poi crescita fluida
t_body = np.linspace(0, 1, body_frames)
n_animated = np.round(5 + (150 - 5) * (t_body**2.5)).astype(int)
n_pause = np.full(pause_frames, n_animated[0])
n_values = np.concatenate([n_pause, n_animated])

# ========================================================================
# 3. SETUP FIGURA
# ========================================================================

fig = plt.figure(figsize=(10, 10), facecolor='white')
gs = fig.add_gridspec(2, 2, height_ratios=[1.8, 1], hspace=0.3, wspace=0.3)

ax_main = fig.add_subplot(gs[0, :])
ax_err = fig.add_subplot(gs[1, 0])
ax_info = fig.add_subplot(gs[1, 1])

history_n = []
history_error = []

# Elementi statici
ax_main.plot(x_smooth, y_smooth, color=COLORS['func_line'], lw=2.5, zorder=5)
ax_main.fill_between(x_smooth, y_smooth, color=COLORS['true_area'], alpha=0.5)
ax_main.set_title(r"$\mathrm{Integrazione\ Numerica:\ Somme\ di\ Riemann}$", fontsize=18, pad=20)
ax_main.set_xlabel(r"$x$")
ax_main.set_ylabel(r"$f(x)$")
ax_main.grid(True, ls='--', alpha=0.3, color='gray')

error_line, = ax_err.plot([], [], color=COLORS['error_line'], lw=1.5)
error_point, = ax_err.plot([], [], 'o', color=COLORS['error_line'], ms=4)
ax_err.set_title(r"$\mathrm{Convergenza\ dell'Errore}$", fontsize=12)
ax_err.set_yscale('log')
ax_err.set_xscale('log')
ax_err.grid(True, which='both', ls=':', alpha=0.4)

ax_info.axis('off')
txt_template = (
    r"$\mathbf{{Parametri}}$" + "\n"
    r"$n = {n}$" + "\n"
    r"$\Delta x = {dx:.4f}$" + "\n\n"
    r"$\mathbf{{Valori}}$" + "\n"
    r"$S_n = {area:.6f}$" + "\n"
    r"$I = {true:.6f}$" + "\n\n"
    r"$\mathbf{{Analisi\ Errore}}$" + "\n"
    r"$\varepsilon = |I - S_n| = {err:.2e}$" + "\n"
    r"$\varepsilon_{{\%}} = {err_perc:.2f}\%$"
)
text_obj = ax_info.text(0.1, 0.95, "", transform=ax_info.transAxes, 
                        va='top', family='serif', fontsize=12, linespacing=1.8)

# AGGIUNTA CREDITI
fig.text(0.95, 0.03, r"$\mathrm{Luca\ Augusto\ Torelli\ -\ Politecnico\ di\ Torino\ (2026)}$", 
         ha='right', fontsize=10, color='#555555', alpha=0.8)

# ========================================================================
# 4. FUNZIONE ANIMAZIONE
# ========================================================================

bars_container = None

def update(frame):
    global bars_container
    
    n = n_values[frame]
    x_rect = np.linspace(a, b, n + 1)
    dx = x_rect[1] - x_rect[0]
    heights = f(x_rect[:-1])
    area_approx = np.sum(heights * dx)
    errore_abs = abs(area_vera - area_approx)
    errore_perc = (errore_abs / area_vera) * 100

    if bars_container:
        for bar in bars_container:
            bar.remove()

    bars_container = ax_main.bar(x_rect[:-1], heights, width=dx, align='edge',
                                 color=COLORS['rect_fill'], edgecolor=COLORS['rect_edge'],
                                 lw=0.5, alpha=0.6)

    # Aggiorna lo storico errore solo se n cambia (evita duplicati nella pausa)
    if frame == 0 or n != n_values[frame-1]:
        history_n.append(n)
        history_error.append(errore_abs)
        error_line.set_data(history_n, history_error)
        error_point.set_data([n], [errore_abs])
        
        ax_err.set_xlim(min(history_n)*0.9, max(history_n)*1.1)
        ax_err.set_ylim(min(history_error)*0.5, max(history_error)*2)

    text_obj.set_text(txt_template.format(
        n=int(n), dx=dx, area=area_approx, true=area_vera,
        err=errore_abs, err_perc=errore_perc
    ))

    return bars_container, error_line, error_point, text_obj

# ========================================================================
# 5. SALVATAGGIO
# ========================================================================

print(f"Rendering: {total_frames} frame ({VIDEO_CONFIG['duration_pause']}s pausa + {VIDEO_CONFIG['duration_body']}s animazione)...")
anim = animation.FuncAnimation(fig, update, frames=total_frames, blit=False)

writer = FFMpegWriter(fps=fps, bitrate=5000)
anim.save('riemann_polito_final.mp4', writer=writer)

print("Video completato con successo.")
plt.close()

Rendering: 420 frame (2s pausa + 12s animazione)...


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter

# ========================================================================
# 1. CONFIGURAZIONE E STILE
# ========================================================================

VIDEO_CONFIG = {'fps': 30, 'dpi': 150, 'duration_sec': 15}

plt.rcParams.update({
    "mathtext.fontset": "cm",
    "font.family": "serif",
    "figure.facecolor": "white",
    "axes.facecolor": "white"
})

COLORS = {
    'func': '#003366', 'rect': '#A0C4FF', 'edge': '#003366',
    'area': '#F5F5F5', 'err': '#B22222'
}

# ========================================================================
# 2. FUNZIONE E CALCOLO AREA
# ========================================================================

def f(x): return np.sin(x) + 0.5 * x
a, b = 0, 5
x_fine = np.linspace(a, b, 1500)
# Calcolo dell'area "vera" (quella che cerchiamo col limite)
area_vera = np.trapezoid(f(x_fine), x_fine)

total_frames = VIDEO_CONFIG['fps'] * VIDEO_CONFIG['duration_sec']
n_values = np.linspace(4, 300, total_frames).astype(int)

# ========================================================================
# 3. SETUP FIGURA
# ========================================================================

fig = plt.figure(figsize=(12, 10), facecolor='white')
gs = fig.add_gridspec(2, 2, height_ratios=[1.8, 1], hspace=0.35, wspace=0.25)

ax_main = fig.add_subplot(gs[0, :])
ax_err = fig.add_subplot(gs[1, 0])
ax_info = fig.add_subplot(gs[1, 1])

# Grafico della Funzione
ax_main.plot(x_fine, f(x_fine), color=COLORS['func'], lw=2.5, zorder=5)
ax_main.fill_between(x_fine, f(x_fine), color=COLORS['area'], alpha=0.8)
ax_main.set_title(r"Integrale Definito come Limite di Somme di Riemann", fontsize=18, pad=20)
ax_main.set_xlabel(r"$x$")
ax_main.set_ylabel(r"$f(x)$")

# Grafico della Precisione
err_line, = ax_err.plot([], [], color=COLORS['err'], lw=2)
err_dot, = ax_err.plot([], [], 'o', color=COLORS['err'], ms=5)
ax_err.set_xscale('log')
ax_err.set_yscale('log')
ax_err.set_title("Riduzione dello Scarto", fontsize=12)
ax_err.set_xlabel("Numero di Rettangoli ($n$)")
ax_err.set_ylabel("Differenza dall'Area Esatta")
ax_err.grid(True, which="both", ls=":", alpha=0.5)

ax_info.axis('off')
text_obj = ax_info.text(0.05, 0.95, "", transform=ax_info.transAxes, 
                        va='top', family='serif', fontsize=12, linespacing=1.8)

# Crediti
fig.text(0.95, 0.02, r"$\mathrm{Luca\ Augusto\ Torelli\ -\ Politecnico\ di\ Torino}$", 
         ha='right', fontsize=9, color='gray')

# ========================================================================
# 4. ANIMAZIONE
# ========================================================================

history_n, history_err = [], []
bars = None

def update(frame):
    global bars
    n = int(n_values[frame])
    dx = (b - a) / n
    x_nodes = np.linspace(a, b, n + 1)
    heights = f(x_nodes[:-1])
    somma_riemann = np.sum(heights * dx)
    scarto = abs(area_vera - somma_riemann)

    if bars: [b.remove() for b in bars]
    bars = ax_main.bar(x_nodes[:-1], heights, width=dx, align='edge',
                       color=COLORS['rect'], edgecolor=COLORS['edge'], 
                       lw=0.5 if n < 70 else 0.1, alpha=0.6)

    if not history_n or n != history_n[-1]:
        history_n.append(n); history_err.append(scarto)
        err_line.set_data(history_n, history_err)
        err_dot.set_data([n], [scarto])
        ax_err.set_xlim(3, 350)
        ax_err.set_ylim(min(history_err)*0.5, max(history_err)*1.5)

    # Testo semplificato per Analisi I
    info_txt = (
        r"$\mathbf{Analisi\ dell'Area}$" + "\n" +
        r"Intervallo: $[0, 5]$" + "\n\n" +
        r"$\mathbf{Costruzione}$" + "\n" +
        r"Num. Rettangoli: $n = " + str(n) + r"$" + "\n" +
        r"Base rettangolo: $\Delta x = " + f"{dx:.4f}" + r"$" + "\n\n" +
        r"$\mathbf{Risultato}$" + "\n" +
        r"Somma Calcolata: $S_n = " + f"{somma_riemann:.6f}" + r"$" + "\n" +
        r"Area Esatta: $I \approx " + f"{area_vera:.6f}" + r"$" + "\n\n" 

    )
    text_obj.set_text(info_txt)
    return bars, err_line, err_dot, text_obj

# ========================================================================
# 5. RENDER
# ========================================================================

print(f"Generazione video per Analisi I...")
anim = animation.FuncAnimation(fig, update, frames=total_frames, blit=False)
writer = FFMpegWriter(fps=VIDEO_CONFIG['fps'], bitrate=5000)
anim.save('riemann_analisi1.mp4', writer=writer)
plt.close()
print("Fatto!")

Generazione video per Analisi I...
Fatto!
